# Exploring the Data for the First Time
## This notebook exclusively depicts all the data without making changes

For figuring out what we want to do with this project

### 6 Data Files
- Bom Movie Gross Info as CSV
- IMDB Database w/ 8 Tables: directors, known_for, movie_akas, move_basics, movie_ratings, person, principals, writers
- Rotten Tomatoes (RT) Movie Info as TSV
- Rotten Tomatoes (RT) Reviews as TSV
- The Movie Database (TMDB) as CSV
- The Numbers (TN) Movie Budgets as CSV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from zipfile import ZipFile # for unzipping the db.zip file
pd.set_option('display.max_columns', None) # show all columns when printing out dataframe

### Opening the Files

### Bom Movie Gross Info

#### The data set comprises of 3387 movies with title, studio, domestic_gross, foreign_gross, release year

In [2]:
bom_gross_df = pd.read_csv('../zippedData/bom.movie_gross.csv.gz')
bom_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
bom_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


## Rotten Tomatoes Movie Info 

#### The data set comprises 1560 movies, which are labeled with an id instead of movie name. 

#### There are 11 columns: id, synopsis, rating, genre, director, writer, theater_date, dvd_date, currency, box_office, runtime, studio

In [4]:
rt_movie_info_df = pd.read_csv('../zippedData/rt.movie_info.tsv.gz', delimiter='\t')
rt_movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [5]:
rt_movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


### Separate dataframe that includes only the rows that have theater_date (and converted to datetime)

In [6]:
rt_movie_info_if_date_exists = rt_movie_info_df.dropna(subset=['theater_date']).copy()
rt_movie_info_if_date_exists['datetime_theater_date'] = pd.to_datetime(rt_movie_info_if_date_exists['theater_date'], format='%b %d, %Y')
rt_movie_info_if_date_exists['datetime_dvd_date'] = pd.to_datetime(rt_movie_info_if_date_exists['dvd_date'], format='%b %d, %Y')


rt_movie_info_if_date_exists[['datetime_theater_date', 'datetime_dvd_date']].head()

,datetime_theater_date,datetime_dvd_date
0,1971-10-09,2001-09-25
1,2012-08-17,2013-01-01
2,1996-09-13,2000-04-18
3,1994-12-09,1997-08-27
5,2000-03-03,2000-07-11


## Rotten Tomatoes Reviews Info 

#### The data set comprises 54432 reviews, with id's that appear to correspond to the movies in rt_movie_info_df. 

#### There are 11 columns: id, review, rating, fresh, critic, top_critic, publisher, date

Converting dates to datetime

In [7]:
rt_reviews_df = pd.read_csv('../zippedData/rt.reviews.tsv.gz', delimiter='\t', encoding='unicode_escape')
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [8]:
# every row has a date in this table, so there is no need to drop any data
rt_reviews_df['datetime_date'] = pd.to_datetime(rt_reviews_df['date'], format='%B %d, %Y')

In [9]:
rt_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             54432 non-null  int64         
 1   review         48869 non-null  object        
 2   rating         40915 non-null  object        
 3   fresh          54432 non-null  object        
 4   critic         51710 non-null  object        
 5   top_critic     54432 non-null  int64         
 6   publisher      54123 non-null  object        
 7   date           54432 non-null  object        
 8   datetime_date  54432 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 3.7+ MB


Number of reviews for each movie id

In [10]:
rt_reviews_df['id'].value_counts()

782     338
1067    275
1525    262
1777    260
1083    260
       ... 
28        1
102       1
348       1
476       1
1727      1
Name: id, Length: 1135, dtype: int64

## The Movie Database Info

#### The data set comprises 26517 rows of movies defined by id and original_title, but there are multiple rows of the same id and/or original_title, so there needs to be some data cleaning to merge information or remove duplicates

#### There are 10 columns: 'Unnamed: 0', genre_ids, id, original_language, original_title, popularity, release_date, title, vote_average, vote_count

#### Additional Notes
- There is an 'Unnamed: 0' column that seems to have been a row id that we can clean up
- The types of some of the columns needs changing
    - genre_ids needs to be converted to a list
    - release_date needs to be datetime

In [11]:
tmdb_movies_df = pd.read_csv('../zippedData/tmdb.movies.csv.gz')
tmdb_movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [12]:
def col_str(col):
    # Input: column (genre_ids) as a string of a list to see if it consists of no genre_id's -> ('[]')
    # Or if it consists of a list of genre ids as a string -> '[#, #, #]'
    # Returns: np.nan if empty list or list of integers if list with values
    
    if col == '[]':
        return np.nan
    else:
        return [int(x) for x in col.replace('[','').replace(']', '').split(',')]

Converting the genre_ids column to a list of genre ids or an np.nan value and then dropping rows that have nan.

We want to only keep rows with genre ids because we want to primarily use that information in deciding what type of movies to make.

In [13]:
tmdb_movies_df['genre_ids'] = tmdb_movies_df['genre_ids'].apply(col_str)
tmdb_movies_df = tmdb_movies_df.dropna(subset=['genre_ids'])

In [14]:
# get list of unique numbers in the genre_ids
all_numbers = []
for x in tmdb_movies_df['genre_ids']:
    all_numbers += x

all_numbers = sorted(list(set(all_numbers)))

In [15]:
# create columns corresponding to the unique genre ids and populate with False
tmdb_movies_df[all_numbers] = False

In [16]:
# If the genre id corresponds to the movie, then the corresponding genre id column is populated with True
for idx, l in enumerate(tmdb_movies_df['genre_ids']):
    for x in l:
        tmdb_movies_df.at[idx, x] = True

In [17]:
# Columns with genre id as the name are converted to the genre name instead
col_rename = {28:'Action', 12:'Adventure', 16:'Animation', 35:'Comedy', 80:'Crime', 99:'Documentary', 
              18:'Drama',10751:'Family', 14:'Fantasy', 36:'History', 27:'Horror', 10402:'Music', 9648:'Mystery',
              10749:'Romance', 878:'Science Fiction', 10770:'TV Movie', 53:'Thriller', 10752:'War', 37:'Western'}

tmdb_movies_df.rename(columns=col_rename, inplace=True)

In [18]:
# release_date is converted to datetime format
tmdb_movies_df['release_date'] = pd.to_datetime(tmdb_movies_df['release_date'], format='%Y-%m-%d')

Printing out the frequency of each genre

In [19]:
tmdb_movies_df[col_rename.values()].sum().astype(int).sort_values(ascending=False)

Drama              8297
Comedy             5652
Documentary        4960
Thriller           4205
Horror             3680
Action             2612
Romance            2321
Science Fiction    1762
Family             1565
Crime              1510
Animation          1486
Adventure          1400
Music              1267
Mystery            1235
Fantasy            1139
TV Movie           1084
History             621
War                 330
Western             205
dtype: int64

In [20]:
tmdb_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26407 entries, 0 to 23797
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Unnamed: 0         24038 non-null  float64       
 1   genre_ids          24038 non-null  object        
 2   id                 24038 non-null  float64       
 3   original_language  24038 non-null  object        
 4   original_title     24038 non-null  object        
 5   popularity         24038 non-null  float64       
 6   release_date       24038 non-null  datetime64[ns]
 7   title              24038 non-null  object        
 8   vote_average       24038 non-null  float64       
 9   vote_count         24038 non-null  float64       
 10  Adventure          24184 non-null  object        
 11  Fantasy            24158 non-null  object        
 12  Animation          24180 non-null  object        
 13  Drama              24888 non-null  object        
 14  Horror

In [21]:
# There seem to be movies repeated or multiple movies with the same title
tmdb_movies_df['original_title'].value_counts()

Eden                      5
Legend                    5
Truth or Dare             5
Lucky                     5
Truth                     4
                         ..
Первый учитель            1
Ghostmates                1
Nick Thune: Thick Noon    1
La pelea de mi vida       1
Red: Werewolf Hunter      1
Name: original_title, Length: 22478, dtype: int64

## The Numbers Movie Budgets Info

#### The data set comprises 5782 rows of movie budgets. There does not appear to be a clear primary key, but the combination of movie and release_date can be used instead. 

#### There are 6 columns: id, release_date, movie, production_budget, domestic_gross, worldwide_gross

#### Additional Notes
- The ids cycle from 1 to 100 then reset as the columns go down
- There are many instances of repeated movie titles (such as King Kong's release from 3 different years)
- movie + release_date are unique identifiers for the rows

In [41]:
tn_movie_budgets = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [42]:
# Converting release_date to datetime type
tn_movie_budgets['datetime_release_date'] = pd.to_datetime(tn_movie_budgets['release_date'], format='%b %d, %Y')

In [43]:
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str[1:].str.replace(',','').astype(int)
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str[1:].str.replace(',','').astype(int)
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str[1:].str.replace(',','').astype(int)

In [44]:
#Created new column 'worldwide profit' by subtracting production_budget from worldwide_gross
tn_movie_budgets['worldwide_profit'] = (tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget'])

# Created new column 'profit_budget_ratio' to show production_budget : worldwide_profit
tn_movie_budgets['ROI'] = (tn_movie_budgets['worldwide_profit'] / tn_movie_budgets['production_budget'])

tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,datetime_release_date,worldwide_profit,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009-12-18,2351345279,5.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011-05-20,635063875,1.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019-06-07,-200237650,-0.572108
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015-05-01,1072413963,3.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017-12-15,999721747,3.153696


In [45]:
# tn_movie_budgets = tn_movie_budgets.sort_values(by=['profit_budget_ratio'], ascending=False)

In [46]:
#removing movie
tn_movie_budgets = tn_movie_budgets[tn_movie_budgets['worldwide_gross'] != 0]

In [47]:
tn_movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,datetime_release_date,worldwide_profit,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009-12-18,2351345279,5.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011-05-20,635063875,1.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019-06-07,-200237650,-0.572108
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015-05-01,1072413963,3.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017-12-15,999721747,3.153696
...,...,...,...,...,...,...,...,...,...
5775,76,"May 26, 2006",Cavite,7000,70071,71644,2006-05-26,64644,9.234857
5776,77,"Dec 31, 2004",The Mongol King,7000,900,900,2004-12-31,-6100,-0.871429
5778,79,"Apr 2, 1999",Following,6000,48482,240495,1999-04-02,234495,39.082500
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,2005-07-13,-3662,-0.732400


In [48]:
tn_budgets_with_tmdb_genres_df = tn_movie_budgets.merge(tmdb_movies_df, how='left', left_on='movie', 
                                                        right_on='original_title')

In [52]:
tn_budgets_with_tmdb_genres_df.head()

,id_x,release_date_x,movie,production_budget,domestic_gross,worldwide_gross,datetime_release_date,worldwide_profit,ROI,Unnamed: 0,genre_ids,id_y,original_language,original_title,popularity,release_date_y,title,vote_average,vote_count,Adventure,Fantasy,Animation,Drama,Horror,Action,Comedy,History,Western,Thriller,Crime,Documentary,Science Fiction,Mystery,Music,Romance,Family,War,TV Movie
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009-12-18,2351345279,5.532577,6.0,"[28, 12, 14, 878]",19995.0,en,Avatar,26.526,2009-12-18,Avatar,7.4,18676.0,True,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011-05-20,635063875,1.546673,2470.0,"[12, 28, 14]",1865.0,en,Pirates of the Caribbean: On Stranger Tides,30.579,2011-05-20,Pirates of the Caribbean: On Stranger Tides,6.4,8571.0,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019-06-07,-200237650,-0.572108,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015-05-01,1072413963,3.243841,14169.0,"[28, 12, 878]",99861.0,en,Avengers: Age of Ultron,44.383,2015-05-01,Avengers: Age of Ultron,7.3,13457.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017-12-15,999721747,3.153696,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# There are either 57 or 58 rows corresponding to each id
tn_budgets_with_tmdb_genres_df['movie'].value_counts()

Legend                  10
The Gift                 8
Home                     8
Truth or Dare            5
Beauty and the Beast     4
                        ..
We Bought a Zoo          1
The Long Riders          1
Shi Yue Wei Cheng        1
The Bank Job             1
The Stewardesses         1
Name: movie, Length: 5339, dtype: int64

## IMDB Database Info

#### The database consists of 8 tables: movie_basics, directors, known_for, movie_akas, movie_ratings, person, principals, writers

In [54]:
data_dirs = ! ls ../zippedData/
data_dirs = list(data_dirs)
if 'im.db' not in data_dirs: # checks to see if im.db is unzipped. Unzips it in zippedData if not
    zip_imdb = ZipFile('../zippedData/im.db.zip', 'r')
    zip_imdb.extractall(path='../zippedData/')

conn = sqlite3.connect('../zippedData/im.db') # getting connector for im.db

In [55]:
pd.read_sql("""SELECT name FROM sqlite_master  
  WHERE type='table';""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [56]:
pd.read_sql(""" SELECT * FROM movie_basics""", conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [57]:
pd.read_sql(""" SELECT * FROM directors""", conn)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [58]:
pd.read_sql(""" SELECT * FROM known_for""", conn)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


In [59]:
pd.read_sql(""" SELECT * FROM movie_akas""", conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [60]:
pd.read_sql(""" SELECT * FROM movie_ratings""", conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [61]:
pd.read_sql(""" SELECT * FROM persons""", conn)

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [62]:
pd.read_sql(""" SELECT * FROM principals""", conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [63]:
pd.read_sql(""" SELECT * FROM writers""", conn)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825
